In [6]:
import os
import sys
import torch
import argparse
import torchvision
import numpy as np
import torch.nn as nn
import scipy.io as sio
import cv2
import pywt
import pywt.data
import matplotlib.image as image

from PIL import Image
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from os.path import join, isdir, abspath, dirname
import matplotlib.pyplot as plt
# Customized import.
from networks import HED
from datasets import BsdsDataset
from utils import Logger, AverageMeter, \
    load_checkpoint, save_checkpoint, load_vgg16_caffe, load_pretrained_caffe


# # Parse arguments.
# parser = argparse.ArgumentParser(description='HED training.')
# # 1. Actions.
# parser.add_argument('--test',             default=False,             help='Only test the model.', action='store_true')
# # 2. Counts.
# parser.add_argument('--train_batch_size', default=1,    type=int,   metavar='N', help='Training batch size.')
# parser.add_argument('--test_batch_size',  default=1,    type=int,   metavar='N', help='Test batch size.')
# parser.add_argument('--train_iter_size',  default=10,   type=int,   metavar='N', help='Training iteration size.')
# parser.add_argument('--max_epoch',        default=40,   type=int,   metavar='N', help='Total epochs.')
# parser.add_argument('--print_freq',       default=500,  type=int,   metavar='N', help='Print frequency.')
# # 3. Optimizer settings.
# parser.add_argument('--lr',               default=1e-6, type=float, metavar='F', help='Initial learning rate.')
# parser.add_argument('--lr_stepsize',      default=1e4,  type=int,   metavar='N', help='Learning rate step size.')
# # Note: Step size is based on number of iterations, not number of batches.
# #   https://github.com/s9xie/hed/blob/94fb22f10cbfec8d84fbc0642b224022014b6bd6/src/caffe/solver.cpp#L498
# parser.add_argument('--lr_gamma',         default=0.1,  type=float, metavar='F', help='Learning rate decay (gamma).')
# parser.add_argument('--momentum',         default=0.9,  type=float, metavar='F', help='Momentum.')
# parser.add_argument('--weight_decay',     default=2e-4, type=float, metavar='F', help='Weight decay.')
# # 4. Files and folders.
# parser.add_argument('--vgg16_caffe',      default='',                help='Resume VGG-16 Caffe parameters.')
# parser.add_argument('--checkpoint',       default='',                help='Resume the checkpoint.')
# parser.add_argument('--caffe_model',      default='',                help='Resume HED Caffe model.')
# parser.add_argument('--output',           default='./output',        help='Output folder.')
# parser.add_argument('--dataset',          default='./data/HED-BSDS', help='HED-BSDS dataset folder.')
# # 5. Others.
# parser.add_argument('--cpu',              default=False,             help='Enable CPU mode.', action='store_true')
# args = parser.parse_args()

# Set device.
device = torch.device('cpu')


In [2]:
def wav_trans_grayscale(images):
    images = torchvision.utils.make_grid(images)
    npimg = images.numpy()
    img_np_right_dim = np.transpose(npimg, (1, 2, 0))
    img_np_right_dim = cv2.cvtColor(img_np_right_dim, cv2.COLOR_BGR2GRAY)
    coeffs2 = pywt.dwt2(img_np_right_dim, 'haar')
    LL, (LH, HL, HH) = coeffs2
    return([LL, LH, HL, HH])

def WT_grayscale_path_input(path):  
    """Takes as input path string"""
    orig_image = plt.imread(path)
    if(len(orig_image.shape)==3):
        gray_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = orig_image
    coeffs2 = pywt.dwt2(gray_image, 'haar')
    LL, (LH, HL, HH) = coeffs2
    return [LL, LH, HL, HH]

def im_save_path_modifier(input_path, mode):
    """
    parameter: input_path is the path of the original image
    parameter: mode is one of LL, LH, HL, HH
    """
    dir_path = input_path[:16] + f'{mode}/' + input_path[16:input_path.rfind('/')]
    image_path = input_path[:16] + f'{mode}/' + input_path[16:]
    return(dir_path, image_path)

def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [3]:
# current_dir = abspath(dirname(__file__))
# output_dir = join(current_dir, args.output)
# if not isdir(output_dir):
#     os.makedirs(output_dir)

# # Set logger.
# now_str = datetime.now().strftime('%y%m%d-%H%M%S')
# log = Logger(join(output_dir, 'log-{}.txt'.format(now_str)))
# sys.stdout = log  # Overwrite the standard output.

################################################
# II. Datasets.
################################################
# Datasets and dataloaders.
train_dataset = BsdsDataset(dataset_dir='./data/HED-BSDS', split='train')
test_dataset  = BsdsDataset(dataset_dir='./data/HED-BSDS', split='test')
train_loader  = DataLoader(train_dataset, batch_size=1,
                           num_workers=4, drop_last=True, shuffle=True)
test_loader   = DataLoader(test_dataset,  batch_size=1,
                           num_workers=4, drop_last=False, shuffle=False)



In [4]:
dataiter = iter(test_loader)

In [5]:
test_dataset.__len__()

200

In [11]:
for j, data in tqdm(enumerate(test_loader, 0)):
    
    images, path = data
    
    print(path[0])

    wav_trans_imlist = WT_grayscale_path_input(path[0])
    wav_trans_imlist_names  = ['LL', 'LH', 'HL', 'HH']


    for i in range(len(wav_trans_imlist)):

        # saving image

        dir_path, image_path = im_save_path_modifier(path[0], str(wav_trans_imlist_names[i]))
        os.makedirs(dir_path, exist_ok=True)
        plt.imsave(image_path, wav_trans_imlist[i], cmap = 'gray')

        
    if(j%100==0):
        print(j)

2it [00:00, 14.33it/s]

./data/HED-BSDS/test/100007.jpg
0
./data/HED-BSDS/test/100039.jpg
./data/HED-BSDS/test/100099.jpg
./data/HED-BSDS/test/10081.jpg


7it [00:00, 16.48it/s]

./data/HED-BSDS/test/101027.jpg
./data/HED-BSDS/test/101084.jpg
./data/HED-BSDS/test/102062.jpg
./data/HED-BSDS/test/103006.jpg


10it [00:00, 18.09it/s]

...

./data/HED-BSDS/test/77062.jpg
./data/HED-BSDS/test/78098.jpg
./data/HED-BSDS/test/79073.jpg
./data/HED-BSDS/test/80085.jpg
./data/HED-BSDS/test/80090.jpg


196it [00:08, 22.58it/s]

./data/HED-BSDS/test/8068.jpg
./data/HED-BSDS/test/81066.jpg
./data/HED-BSDS/test/81090.jpg
./data/HED-BSDS/test/81095.jpg
./data/HED-BSDS/test/87015.jpg


200it [00:08, 23.29it/s]

./data/HED-BSDS/test/92014.jpg
./data/HED-BSDS/test/94095.jpg
./data/HED-BSDS/test/97010.jpg


In [10]:
im_save_path_modifier(path[0],'LL')

('./data/HED-BSDS/LL/test', './data/HED-BSDS/LL/test/97010.jpg')